In [63]:
import ast
import astor
import random

# Define possible variable names
VARIABLES = ["a", "b", "c", "d", "e"]
DIGITS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
VARIABLE_SET = VARIABLES + DIGITS + ["_", ""]

def generate_random_variable():
    return random.choice(VARIABLES) + random.choice(DIGITS)

def should_use_variable():
    return random.choice([True, False])

class ForToWhileTransformer(ast.NodeTransformer):
    def __init__(self):
        self.var_map = {}
        super().__init__()

    def get_variable(self, original):
        if original not in self.var_map:
            new_var = generate_random_variable()
            while new_var in self.var_map.values():
                new_var = generate_random_variable()
            self.var_map[original] = new_var
        return self.var_map[original]

    def visit_For(self, node):
        # Ensure that the loop is a for loop with range
        if isinstance(node.iter, ast.Call) and node.iter.func.id == 'range':
            # Extract range parameters
            range_args = node.iter.args
            start = astor.to_source(range_args[0]).strip().replace('(', '').replace(')', '')
            stop = astor.to_source(range_args[1]).strip().replace('(', '').replace(')', '')
            step = astor.to_source(range_args[2]).strip() if len(range_args) > 2 else '1'
            
            step = step.replace('(', '').replace(')', '')
            step = int(step) if step != '1' else 1  # Convert step to integer for use

            # Generate random variable names
            start_var = generate_random_variable()
            stop_var = generate_random_variable()
            step_var = generate_random_variable()

            # Create a while loop equivalent with variable declarations
            while_loop = ast.While(
                test=ast.Compare(left=ast.Name(id=start_var, ctx=ast.Load()), ops=[ast.Lt()], comparators=[ast.Name(id=stop_var, ctx=ast.Load())]),
                body=[
                    ast.Expr(value=ast.Call(func=ast.Name(id='print', ctx=ast.Load()), args=[ast.Name(id=start_var, ctx=ast.Load())], keywords=[])),
                    ast.Assign(targets=[ast.Name(id=start_var, ctx=ast.Store())], value=ast.BinOp(left=ast.Name(id=start_var, ctx=ast.Load()), op=ast.Add(), right=ast.Constant(value=step)))
                ],
                orelse=[]
            )
            
            # Replace the for loop with the while loop
            return [ast.Assign(targets=[ast.Name(id=start_var, ctx=ast.Store())], value=ast.Constant(value=int(start))),
                    ast.Assign(targets=[ast.Name(id=stop_var, ctx=ast.Store())], value=ast.Constant(value=int(stop))),
                    ast.Assign(targets=[ast.Name(id=step_var, ctx=ast.Store())], value=ast.Constant(value=step)),
                    while_loop]
        return node

def transform_code(code):
    tree = ast.parse(code)
    transformer = ForToWhileTransformer()
    transformed_tree = transformer.visit(tree)
    return astor.to_source(transformed_tree)

# Original snippet with a for loop
original_code = """
e = 5
for e in range(4, 10):
    print(e)
"""

# Transform the code
transformed_code = transform_code(original_code)
print(transformed_code)


e = 5
d0 = 4
e4 = 10
c2 = 1
while d0 < e4:
    print(d0)
    d0 = d0 + 1

